In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Before beginning I want to thank:
## @teckmengwong for his [notebook](https://www.kaggle.com/teckmengwong/extra-blender)

## What is pseudolabeling?
### Pseudo labeling according to @cdeotte is the process of adding confident predicted test data to your training data. The blended notebook selected in our case has 0.99016 accuracy which means 99016 out of 100000 were predicted correctly. And if we download the csv file with solution of test data to train data, our model will have more data to train and hence better generalization

Here is the [test labels csv](https://www.kaggle.com/ebrahimhaquebhatti/tpsfeb-target)

## **Necessary Imports**

In [ ]:
# !pip install scikit-learn-intelex

# from sklearnex import patch_sklearn
# patch_sklearn()

In [ ]:
%%capture
!pip install pycaret[full]
from pycaret.classification import *
import pandas as pd
import numpy as np

In [ ]:
ef=pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/test.csv')

In [ ]:
pseudo_labels=pd.read_csv('/kaggle/input/tpsfeb-target/target_csv.csv')
pseudo_labels=pseudo_labels.drop(['row_id'],axis=1)

In [ ]:
df=pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/train.csv')

In [ ]:
df

In [ ]:
ef

## Merge pseudo labels with the test set

In [ ]:
result = pd.concat([pseudo_labels, ef], axis=1)
result.reset_index(inplace=True, drop=True)

## Concatenate Train and Test Set

In [ ]:
df_final = pd.concat([df, result], axis=0,ignore_index=True)


In [ ]:
df_final

## Preprocessing using pycaret

In [ ]:
    #setting up the pyCaret regression algorithm
reg = setup(data = df_final,
            target = 'target',
            train_size = 0.7, #70:30 train/validation split
            normalize=True,
            use_gpu = True, #use GPU acceleration to train models
            silent = True, #removes need for confirmation step
            fold = 10, #number of cross-fold validation folds
            n_jobs = -1) #use all processor threads

In [ ]:
a=get_config('y_train')
b=get_config('y_test')

In [ ]:
b.value_counts()

In [ ]:
a.value_counts()

## Prediction using xgboost and Extra Tress

In [ ]:
et=create_model('et')

In [ ]:
xgboost=create_model('xgboost')

## Blending extra trees and xgboost

In [ ]:
blend_specific = blend_models(estimator_list = [xgboost,et])


## Decision Boundary

In [ ]:
plot_model(blend_specific, plot='boundary')

## Predicting Test Data

In [ ]:
predictions=predict_model(blend_specific,data=ef)

In [ ]:
output= pd.DataFrame ({'row_id': ef.row_id, 'Target': predictions.Label})
output.to_csv('submission.csv',index=False)